In [104]:
import pandas as pd
import numpy as np
import re
from string import ascii_letters, digits
from utils import cleaning_dicts
import os

In [105]:
#print(cleaning_dicts.clean_team_pff)
print(cleaning_dicts.clean_team_pff_full)

{'arizona-cardinals': 'ari', 'atlanta-falcons': 'atl', 'baltimore-ravens': 'bal', 'buffalo-bills': 'buf', 'carolina-panthers': 'car', 'chicago-bears': 'chi', 'cincinnati-bengals': 'cin', 'cleveland-browns': 'cle', 'dallas-cowboys': 'dal', 'denver-broncos': 'den', 'detroit-lions': 'det', 'green-bay-packers': 'gb', 'houston-texans': 'hou', 'indianapolis-colts': 'ind', 'jacksonville-jaguars': 'jax', 'kansas-city-chiefs': 'kc', 'los-angeles-chargers': 'lac', 'los-angeles-rams': 'lar', 'miami-dolphins': 'mia', 'minnesota-vikings': 'min', 'new-england-patriots': 'ne', 'new-orleans-saints': 'no', 'new-york-giants': 'nyg', 'new-york-jets': 'nyj', 'oakland-raiders': 'lv', 'philadelphia-eagles': 'phi', 'pittsburgh-steelers': 'pit', 'san-francisco-49ers': 'sf', 'seattle-seahawks': 'sea', 'tampa-bay-buccaneers': 'tb', 'tennessee-titans': 'ten', 'washington-redskins': 'was', 'washington-football-team': 'was', 'las-vegas-raiders': 'lv', 'washington-commanders': 'was', 'washingtoncommanders': 'was', 

In [106]:
cur_dir = os.path.abspath(os.getcwd())
cur_week_folder = 'week_6'
cur_week = '6'
cur_week_int = 6
cur_year_int = 2022

In [107]:
print(cur_dir)

/home/tomb/nfl_models


In [139]:
####################################################################################
                ###   Read-in and clean all passing datasets ###
####################################################################################


passing_depth = pd.read_csv('./scripts/nfl_all/passing_depth_2022.csv')
passing_allowed_pressure = pd.read_csv('./scripts/nfl_all/passing_allowed_pressure_2022.csv')
passing_pressure = pd.read_csv('./scripts/nfl_all/passing_pressure_2022.csv')
passing_concept = pd.read_csv('./scripts/nfl_all/passing_concept_2022.csv')
time_in_pocket = pd.read_csv('./scripts/nfl_all/time_in_pocket_2022.csv')
passing_summ_conc = pd.read_csv('./scripts/nfl_all/passing_summ_conc_2022.csv')



def drop_non_qbs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df=df[df['position'] == 'QB']
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
#     df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
#     df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df
    
passing_depth = drop_non_qbs(passing_depth)
passing_allowed_pressure = drop_non_qbs(passing_allowed_pressure)
passing_pressure = drop_non_qbs(passing_pressure)
passing_concept = drop_non_qbs(passing_concept)
time_in_pocket = drop_non_qbs(time_in_pocket)
passing_summ_conc = drop_non_qbs(passing_summ_conc)


passing_depth = passing_depth[passing_depth.columns.drop(list(passing_depth.filter(regex='left|right|center')))]

####################################################################################
				###   Read-in and clean all receiving datasets ### scripts/nfl_all
####################################################################################
                                 
rec_summ_conc = pd.read_csv('./scripts/nfl_all/rec_summ_conc_2022.csv')
receiving_concept = pd.read_csv('./scripts/nfl_all/receiving_concept_2022.csv')
receiving_depth = pd.read_csv('./scripts/nfl_all/receiving_depth_2022.csv')
receiving_scheme = pd.read_csv('./scripts/nfl_all/receiving_scheme_2022.csv')
                                 
def drop_non_recs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|TE|HB|FB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
#     df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
#     df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df

rec_summ_conc = drop_non_recs(rec_summ_conc)
receiving_concept = drop_non_recs(receiving_concept)
receiving_depth = drop_non_recs(receiving_depth)
receiving_scheme = drop_non_recs(receiving_scheme)


####################################################################################
				###   Read-in and clean all rushing datasets ###
####################################################################################

rush_summ_conc = pd.read_csv('./scripts/nfl_all/rush_summ_conc_2022.csv')                                 


def drop_non_rbs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|HB|FB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
#     df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
#     df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df

rush_summ_conc = drop_non_rbs(rush_summ_conc)


####################################################################################
				###   Read-in and clean all blocking datasets ###
####################################################################################

                               
block_summ_conc = pd.read_csv('./scripts/nfl_all/block_summ_conc_2022.csv')
offense_pass_blocking = pd.read_csv('./scripts/nfl_all/offense_pass_blocking_2022.csv')
offense_run_blocking = pd.read_csv('./scripts/nfl_all/offense_run_blocking_2022.csv')                                 


def drop_non_ols(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df = df[df['position'].notna()]
    df= df[df.position.str.match('T|C|G|TE')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
#     df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
#     df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df


block_summ_conc	= drop_non_ols(block_summ_conc)
offense_pass_blocking = drop_non_ols(offense_pass_blocking)
offense_run_blocking = drop_non_ols(offense_run_blocking)



####################################################################################
				###   Read-in and clean all defensive datasets ###
####################################################################################

                                
def_summ_conc = pd.read_csv('./scripts/nfl_all/def_summ_conc_2022.csv')
pass_rush_summary = pd.read_csv('./scripts/nfl_all/pass_rush_summary_2022.csv')
run_defense_summary = pd.read_csv('./scripts/nfl_all/run_defense_summary_2022.csv')
defense_coverage_scheme = pd.read_csv('./scripts/nfl_all/defense_coverage_scheme_2022.csv')
defense_coverage_summary = pd.read_csv('./scripts/nfl_all/defense_coverage_summary_2022.csv')
slot_coverage = pd.read_csv('./scripts/nfl_all/slot_coverage_2022.csv')

                              
def drop_non_def(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])


#     df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
#     df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df

def_summ_conc = drop_non_def(def_summ_conc)
pass_rush_summary = drop_non_def(pass_rush_summary)
run_defense_summary = drop_non_def(run_defense_summary)
defense_coverage_scheme = drop_non_def(defense_coverage_scheme)
defense_coverage_summary = drop_non_def(defense_coverage_summary)
slot_coverage = drop_non_def(slot_coverage)

def_summ_conc=def_summ_conc[def_summ_conc['position'].isin(["ed","lb","di","s","cb"])]
pass_rush_summary=pass_rush_summary[pass_rush_summary['position'].isin(["ed","lb","di","s"])]
run_defense_summary=run_defense_summary[run_defense_summary['position'].isin(["ed","lb","di","s","cb"])]
defense_coverage_scheme=defense_coverage_scheme[defense_coverage_scheme['position'].isin(["lb","s","cb"])]
defense_coverage_summary=defense_coverage_summary[defense_coverage_summary['position'].isin(["lb","s","cb"])]
slot_coverage=slot_coverage[slot_coverage['position'].isin(["lb","s","cb"])]

####################################################################################
				###   Read-in and clean all special teams datasets ###
####################################################################################	


st_kickers = pd.read_csv('./scripts/nfl_all/st_kickers_2022.csv')
st_punters = pd.read_csv('./scripts/nfl_all/st_punters_2022.csv')                                 
                                 
                               
def clean_spec(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])


#     df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
#     df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
#     df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df

st_kickers =clean_spec(st_kickers)
st_punters = clean_spec(st_punters)



####################################################################################
####################################################################################
####################################################################################

/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:61: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:96: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:131: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:171: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:214: FutureWarni

In [140]:
pass_rush_summary.head()

,player,numeric_id,position,team_name,player_game_count,batted_passes,declined_penalties,franchise_id,grades_pass_rush_defense,hits,...,true_pass_set_pass_rush_percent,true_pass_set_pass_rush_win_rate,true_pass_set_pass_rush_wins,true_pass_set_prp,true_pass_set_sacks,true_pass_set_snap_counts_pass_play,true_pass_set_snap_counts_pass_rush,true_pass_set_total_pressures,week,year
0,calaiscampbell,4364,di,bal,1,0,0,3,66.5,1,...,100.0,27.3,3,13.6,1,13,13,2,1,2022
1,tysonalualu,5535,di,pit,1,0,0,25,56.3,0,...,100.0,0.0,0,0.0,0,11,11,0,1,2022
2,brandongraham,5538,ed,phi,1,0,0,24,63.9,0,...,100.0,37.5,3,18.8,1,8,8,2,1,2022
3,jerryhughes,5556,ed,hou,1,0,0,13,86.6,0,...,94.4,29.4,5,5.9,0,18,17,2,1,2022
4,carlosdunlap,5579,ed,kc,1,0,0,16,55.9,0,...,91.7,10.0,1,10.0,1,12,11,1,1,2022


In [141]:

passing_depth = passing_depth[passing_depth['week'] >= cur_week_int-2]
passing_allowed_pressure = passing_allowed_pressure[passing_allowed_pressure['week'] >= cur_week_int-2]
passing_pressure = passing_pressure[passing_pressure['week'] >= cur_week_int-2]
passing_concept = passing_concept[passing_concept['week'] >= cur_week_int-2]
time_in_pocket = time_in_pocket[time_in_pocket['week'] >= cur_week_int-2]
passing_summ_conc = passing_summ_conc[passing_summ_conc['week'] >= cur_week_int-2]


rec_summ_conc = rec_summ_conc[rec_summ_conc['week'] >= cur_week_int-2]
receiving_concept =receiving_concept[receiving_concept['week'] >= cur_week_int-2]
receiving_depth = receiving_depth[receiving_depth['week'] >= cur_week_int-2]
receiving_scheme = receiving_scheme[receiving_scheme['week'] >= cur_week_int-2]

rush_summ_conc =rush_summ_conc[rush_summ_conc['week'] >= cur_week_int-2]

block_summ_conc = block_summ_conc[block_summ_conc['week'] >= cur_week_int-2]
offense_pass_blocking = offense_pass_blocking[offense_pass_blocking['week'] >= cur_week_int-2]
offense_run_blocking = offense_run_blocking[offense_run_blocking['week'] >= cur_week_int-2]

def_summ_conc = def_summ_conc[def_summ_conc['week'] >= cur_week_int-2]
pass_rush_summary = pass_rush_summary[pass_rush_summary['week'] >= cur_week_int-2]
run_defense_summary = run_defense_summary[run_defense_summary['week'] >= cur_week_int-2]
defense_coverage_scheme = defense_coverage_scheme[defense_coverage_scheme['week'] >= cur_week_int-2]
defense_coverage_summary = defense_coverage_summary[defense_coverage_summary['week'] >= cur_week_int-2]
slot_coverage = slot_coverage[slot_coverage['week'] >= cur_week_int-2]

st_kickers = st_kickers[st_kickers['week'] >= cur_week_int-2]
st_punters = st_punters[st_punters['week'] >= cur_week_int-2]


In [142]:
passing_depth = passing_depth.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
passing_allowed_pressure = passing_allowed_pressure.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
passing_pressure = passing_pressure.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
passing_concept = passing_concept.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
time_in_pocket = time_in_pocket.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
passing_summ_conc = passing_summ_conc.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)


rec_summ_conc = rec_summ_conc.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
receiving_concept =receiving_concept.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
receiving_depth = receiving_depth.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
receiving_scheme = receiving_scheme.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)

rush_summ_conc =rush_summ_conc.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)

block_summ_conc = block_summ_conc.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
offense_pass_blocking = offense_pass_blocking.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
offense_run_blocking = offense_run_blocking.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)

def_summ_conc = def_summ_conc.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
pass_rush_summary = pass_rush_summary.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
run_defense_summary = run_defense_summary.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
defense_coverage_scheme = defense_coverage_scheme.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
defense_coverage_summary = defense_coverage_summary.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
slot_coverage = slot_coverage.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)

st_kickers = st_kickers.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
st_punters = st_punters.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)

In [143]:
def df_nans(nData):
    a = np.full([nData.shape[0], nData.shape[1]], 0)
    np_df = pd.DataFrame(a, columns=nData.columns)
    ids = nData.iloc[:,:8].reset_index(drop=True)
    other_cols = np_df.iloc[:,8:]
    nData = pd.concat([ids, other_cols], axis=1)
    
    nData.replace(["NaN", 'NaT'], np.nan, inplace = True)
    nData=nData[pd.notnull(nData['player'])]
    #nData=nData.dropna(thresh=10)
    nData['year'] = cur_year_int
    nData['week'] = cur_week_int
    nData['plyr_number'] = cur_week_int
    nData['year'] = nData['year'].astype(int)
    nData['week'] = nData['week'].astype(int)
    return nData


passing_depth_new = df_nans(passing_depth_new)
passing_allowed_pressure_new = df_nans(passing_allowed_pressure_new)
passing_pressure_new = df_nans(passing_pressure_new)
passing_concept_new = df_nans(passing_concept_new)
time_in_pocket_new = df_nans(time_in_pocket_new)
passing_summ_conc_new = df_nans(passing_summ_conc_new)


rec_summ_conc = df_nans(rec_summ_conc)
receiving_concept =df_nans(receiving_concept)
receiving_depth = df_nans(receiving_depth)
receiving_scheme = df_nans(receiving_scheme)

rush_summ_conc = df_nans(rush_summ_conc)

block_summ_conc = df_nans(block_summ_conc)
offense_pass_blocking = df_nans(offense_pass_blocking)
offense_run_blocking = df_nans(offense_run_blocking)

def_summ_conc = df_nans(def_summ_conc)
pass_rush_summary = df_nans(pass_rush_summary)
run_defense_summary = df_nans(run_defense_summary)
defense_coverage_scheme = df_nans(defense_coverage_scheme)
defense_coverage_summary = df_nans(defense_coverage_summary)
slot_coverage = df_nans(slot_coverage)

st_kickers = df_nans(st_kickers)
st_punters = df_nans(st_punters)


In [144]:
nfl_injuries = pd.read_csv('../nfl_models/current_data/week_'+cur_week+'/nfl_injuries.csv')
nfl_injuries

,Player,Team,Pos,Injury,Status,Est. Return
0,Jamal Adams,SEA,S,Knee,IR,Subscribers Only
1,Matt Adams,CHI,LB,Calf,IR,Subscribers Only
2,Montravius Adams,PIT,DT,Hip,Questionable,Subscribers Only
3,Rodney Adams,NYJ,WR,Illness,NFI-R,Subscribers Only
4,Olasunkanmi Adeniyi,TEN,LB,Neck,Questionable,Subscribers Only
...,...,...,...,...,...,...
743,Nate Wozniak,FA,OT,Undisclosed,Reserve-Ret,Subscribers Only
744,Devonte Wyatt,GB,DE,Quadriceps,Questionable,Subscribers Only
745,Chase Young,WAS,DE,Knee - ACL + MCL,PUP-R,Subscribers Only
746,Dareke Young,SEA,WR,Quadriceps,Questionable,Subscribers Only


In [145]:
def clean_name(df):
    result = ''.join(c for c in df['player'] if c.isalpha())
    return result.lower()
# file:///home/tomb/nfl_models/current_data/week_6/nfl_injuries.csv


nfl_injuries = pd.read_csv('../nfl_models/current_data/week_'+cur_week+'/nfl_injuries.csv')
nfl_injuries = nfl_injuries[['Player','Team','Pos','Injury','Status']]
nfl_injuries.columns = ['Player','Tm','Pos','Type','Class']
nfl_injuries.columns = [x.lower() for x in nfl_injuries.columns]
nfl_injuries = nfl_injuries[nfl_injuries['tm'] != 'FA']
nfl_injuries.drop_duplicates(subset=['player','tm','pos'], keep='first', inplace=True)
nfl_injuries['player'] = nfl_injuries.apply(lambda df: clean_name(df), axis=1)


def clean_non_player_dfs(df):
##  basic scrubbing to clean data ##
    df=df.apply(lambda x: x.astype(str).str.lower())
    df=df.replace('-','', regex=True)
    df['player']=df['player'].replace(' ','', regex=True)
    df['tm']=df['tm'].replace(' ','', regex=True)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['tm'].map(cleaning_dicts.clean_team_pfr).fillna(df['tm'])
    df['pos'] = df['pos'].map(cleaning_dicts.pos_dict).fillna(df['pos'])

    df.insert(0, "player_team", (df['player']+'_'+df['team_name']))
    return df

nfl_injuries = clean_non_player_dfs(nfl_injuries)
nfl_injuries['class']=nfl_injuries['class'].str.replace('pupr','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('nfir','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('nfia','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('inactive','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reservecovid19','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reservesus','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reservednr','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reserveret','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reservesus','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reserveex','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('pupp','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('irr','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('ir','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('doubtful','out')

nfl_injuries['class'].value_counts()

out             280
questionable    167
Name: class, dtype: int64

In [146]:
pass_rush_summary

,player,numeric_id,position,team_name,player_game_count,batted_passes,declined_penalties,franchise_id,grades_pass_rush_defense,hits,...,true_pass_set_pass_rush_win_rate,true_pass_set_pass_rush_wins,true_pass_set_prp,true_pass_set_sacks,true_pass_set_snap_counts_pass_play,true_pass_set_snap_counts_pass_rush,true_pass_set_total_pressures,week,year,plyr_number
0,nickbellore,6515,lb,sea,1,0,0,29,0,0,...,0,0,0,0,0,0,0,6,2022,6
1,harrisonsmith,7641,s,min,1,0,1,18,0,0,...,0,0,0,0,0,0,0,6,2022,6
2,johnathanhankins,7830,di,lv,1,0,0,23,0,0,...,0,0,0,0,0,0,0,6,2022,6
3,jordanpoyer,8000,s,buf,1,0,0,4,0,0,...,0,0,0,0,0,0,0,6,2022,6
4,christiankirksey,8706,lb,hou,1,0,0,13,0,0,...,0,0,0,0,0,0,0,6,2022,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,samokuayinonu,101425,ed,ten,1,0,0,31,0,0,...,0,0,0,0,0,0,0,6,2022,6
472,leochenal,101463,lb,kc,1,0,0,16,0,0,...,0,0,0,0,0,0,0,6,2022,6
473,rodneythomasii,108389,s,ind,1,0,0,14,0,0,...,0,0,0,0,0,0,0,6,2022,6
474,perrionwinfrey,122952,di,cle,1,1,0,8,0,0,...,0,0,0,0,0,0,0,6,2022,6


In [147]:
passing_depth_new['player']= passing_depth_new.apply(lambda df: clean_name(df), axis=1)
passing_allowed_pressure_new['player']= passing_allowed_pressure_new.apply(lambda df: clean_name(df), axis=1)
passing_pressure_new['player']= passing_pressure_new.apply(lambda df: clean_name(df), axis=1)
passing_concept_new['player']= passing_concept_new.apply(lambda df: clean_name(df), axis=1)
time_in_pocket_new['player']= time_in_pocket_new.apply(lambda df: clean_name(df), axis=1)
passing_summ_conc_new['player']= passing_summ_conc_new.apply(lambda df: clean_name(df), axis=1)


rec_summ_conc['player']= rec_summ_conc.apply(lambda df: clean_name(df), axis=1)
receiving_concept['player']=receiving_concept.apply(lambda df: clean_name(df), axis=1)
receiving_depth['player']= receiving_depth.apply(lambda df: clean_name(df), axis=1)
receiving_scheme['player']= receiving_scheme.apply(lambda df: clean_name(df), axis=1)

rush_summ_conc['player']=rush_summ_conc.apply(lambda df: clean_name(df), axis=1)

block_summ_conc['player']= block_summ_conc.apply(lambda df: clean_name(df), axis=1)
offense_pass_blocking['player']= offense_pass_blocking.apply(lambda df: clean_name(df), axis=1)
offense_run_blocking['player']= offense_run_blocking.apply(lambda df: clean_name(df), axis=1)

def_summ_conc['player']= def_summ_conc.apply(lambda df: clean_name(df), axis=1)
pass_rush_summary['player']= pass_rush_summary.apply(lambda df: clean_name(df), axis=1)
run_defense_summary['player']= run_defense_summary.apply(lambda df: clean_name(df), axis=1)
defense_coverage_scheme['player']= defense_coverage_scheme.apply(lambda df: clean_name(df), axis=1)
defense_coverage_summary['player']= defense_coverage_summary.apply(lambda df: clean_name(df), axis=1)
slot_coverage['player']= slot_coverage.apply(lambda df: clean_name(df), axis=1)

st_kickers['player']= st_kickers.apply(lambda df: clean_name(df), axis=1)
st_punters['player']= st_punters.apply(lambda df: clean_name(df), axis=1)



In [18]:
def drop_non_recs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|TE|HB|FB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df['week'] = df['week'].astype(int)
    return df


def clean_new_week(df):
##  basic scrubbing to clean data ##
    df['year'] = df['year'].apply(str)
    df=df.apply(lambda x: x.astype(str).str.lower())
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)

    ##  create our unique ids  ##
    df.insert(0, "unique_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['plyr_number']))
    df.insert(1, "player_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(2, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['plyr_number']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(4, "player_name", (df['player']))
    df.insert(5, "player_team", (df['player']+'_'+df['team_name']))
    return df

rush_std_new = clean_new_week(rush_std_new)
qb_standard_new =clean_new_week(qb_standard_new)
oline_standard_new=clean_new_week(oline_standard_new)
rec_std_new=clean_new_week(rec_std_new)
def_standard_new=clean_new_week(def_standard_new)



In [19]:
rush_std_new.head().T

,0,1,2,3,4
unique_id,dalvincook_min_2021_17,tylerhuntley_bal_2021_17,austinekeler_lac_2021_17,taysomhill_no_2021_17,rhamondrestevenson_ne_2021_17
player_id,dalvincook_min_2021,tylerhuntley_bal_2021,austinekeler_lac_2021,taysomhill_no_2021,rhamondrestevenson_ne_2021
unique_team_id,min_2021_17,bal_2021_17,lac_2021_17,no_2021_17,ne_2021_17
team_id_impute,min_2021,bal_2021,lac_2021,no_2021,ne_2021
player_name,dalvincook,tylerhuntley,austinekeler,taysomhill,rhamondrestevenson
player_team,dalvincook_min,tylerhuntley_bal,austinekeler_lac,taysomhill_no,rhamondrestevenson_ne
player,dalvincook,tylerhuntley,austinekeler,taysomhill,rhamondrestevenson
position,hb,qb,hb,qb,hb
team_name,min,bal,lac,no,ne
player_game_count,1,1,1,1,1


In [20]:
## Add this for dummy adj comp percentage columns
#qb_standard_new['adj_perc'] = 0
#qb_standard_new['qb_adjform'] = 0

In [21]:


from fuzzywuzzy import fuzz

def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = 0
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

def fuzzy_inj_match(df):
    # List for dicts for easy dataframe creation
    dict_list = []
    # iterating over our players without salaries found above
    for name in df.player_team:
        # Use our method to find best match, we can set a threshold here
        match = match_name(name, nfl_injuries.player_team, 85)

        # New dict for storing data
        dict_ = {}
        dict_.update({"player_name" : name})
        dict_.update({"match_name" : match[0]})
        dict_.update({"score" : match[1]})
        dict_list.append(dict_)
    return pd.DataFrame(dict_list)
        #return pd.merge(df, merge_table, left_on='unique_player_id', right_on='player_name', how='left')

# Display results
rush_std_new_match = fuzzy_inj_match(rush_std_new)
qb_standard_new_match = fuzzy_inj_match(qb_standard_new)
oline_standard_new_match = fuzzy_inj_match(oline_standard_new)
rec_std_new_match = fuzzy_inj_match(rec_std_new)
def_standard_new_match = fuzzy_inj_match(def_standard_new)

/home/tom/anaconda3/envs/spreads/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [22]:
rush_std_new_match = rush_std_new_match[rush_std_new_match['score'] != 0]
qb_standard_new_match = qb_standard_new_match[qb_standard_new_match['score'] != 0]
oline_standard_new_match = oline_standard_new_match[oline_standard_new_match['score'] != 0]
rec_std_new_match = rec_std_new_match[rec_std_new_match['score'] != 0]
def_standard_new_match = def_standard_new_match[def_standard_new_match['score'] != 0]

In [23]:
def_standard_new_match.head(n=20)

,player_name,match_name,score
1,jasonpierrepaul_tb,jasonpierrepaul_tb,100
2,andrewsendejo_ind,andrewsendejo_ind,100
5,lavontedavid_tb,lavontedavid_tb,100
7,akiemhicks_chi,akiemhicks_chi,100
10,johnathanhankins_lv,johnathanhankins_lv,100
12,lerenteemccray_jax,lerenteemccray_jax,100
16,dannyshelton_nyg,dannyshelton_nyg,100
17,landoncollins_was,landoncollins_was,100
19,jordanphillips_ari,jordanphillips_ari,100
26,deshazoreverett_was,deshazoreverett_was,100


In [24]:
nfl_injuries = nfl_injuries[['player_team','pos','type','class']]
rush_std_new_match = pd.merge(rush_std_new_match, nfl_injuries, left_on='player_name', right_on='player_team', how='left')
qb_standard_new_match = pd.merge(qb_standard_new_match, nfl_injuries, left_on='player_name', right_on='player_team', how='left')
oline_standard_new_match = pd.merge(oline_standard_new_match, nfl_injuries, left_on='player_name', right_on='player_team', how='left')
rec_std_new_match = pd.merge(rec_std_new_match, nfl_injuries, left_on='player_name', right_on='player_team', how='left')
def_standard_new_match = pd.merge(def_standard_new_match, nfl_injuries, left_on='player_name', right_on='player_team', how='left')


In [25]:
rec_std_new_match['class'].value_counts()

out             28
questionable    18
ir               9
doubtful         1
Name: class, dtype: int64

In [26]:
rush_std_new_match = rush_std_new_match.loc[(rush_std_new_match['class'] == 'ir') | (rush_std_new_match['class'] == 'out') | (rush_std_new_match['class'] == 'bench') | (rush_std_new_match['class'] == 'doubtful') | (rush_std_new_match['class'] == 'cut') | (rush_std_new_match['class'] == 'reserve-covid-19')]#Reserve-Covid-19
qb_standard_new_match = qb_standard_new_match.loc[(qb_standard_new_match['class'] == 'ir') | (qb_standard_new_match['class'] == 'out') | (qb_standard_new_match['class'] == 'bench') | (qb_standard_new_match['class'] == 'doubtful') | (qb_standard_new_match['class'] == 'cut') | (qb_standard_new_match['class'] == 'reserve-covid-19')]
oline_standard_new_match = oline_standard_new_match.loc[(oline_standard_new_match['class'] == 'ir') | (oline_standard_new_match['class'] == 'out') | (oline_standard_new_match['class'] == 'bench') | (oline_standard_new_match['class'] == 'doubtful') | (oline_standard_new_match['class'] == 'cut') | (oline_standard_new_match['class'] == 'reserve-covid-19')]
rec_std_new_match = rec_std_new_match.loc[(rec_std_new_match['class'] == 'ir') | (rec_std_new_match['class'] == 'out') | (rec_std_new_match['class'] == 'bench') | (rec_std_new_match['class'] == 'doubtful') | (rec_std_new_match['class'] == 'cut') | (rec_std_new_match['class'] == 'reserve-covid-19')]
def_standard_new_match = def_standard_new_match.loc[(def_standard_new_match['class'] == 'ir') | (def_standard_new_match['class'] == 'out') | (def_standard_new_match['class'] == 'bench') | (def_standard_new_match['class'] == 'doubtful') | (def_standard_new_match['class'] == 'cut') | (def_standard_new_match['class'] == 'reserve-covid-19')]

In [27]:
rec_std_new_match['class'].value_counts()

out         28
ir           9
doubtful     1
Name: class, dtype: int64

In [28]:
injured_players_list= pd.concat([rush_std_new_match,qb_standard_new_match,oline_standard_new_match,rec_std_new_match,def_standard_new_match], axis=0)

In [29]:
injured_players_list.head(n=100)

,player_name,match_name,score,player_team,pos,type,class
0,leonardfournette_tb,leonardfournette_tb,100,leonardfournette_tb,rb,hamstring,ir
3,davidjohnson_hou,davidjohnson_hou,100,davidjohnson_hou,rb,undisclosed,out
4,teddybridgewater_den,teddybridgewater_den,100,teddybridgewater_den,qb,concussion,out
8,tevincoleman_nyj,tevincoleman_nyj,100,tevincoleman_nyj,rb,undisclosed,out
10,clydeedwardshelaire_kc,clydeedwardshelaire_kc,100,clydeedwardshelaire_kc,rb,shoulder,out
...,...,...,...,...,...,...,...
6,trequansmith_no,trequansmith_no,100,trequansmith_no,wr,chest,out
9,jaelondarden_tb,jaelondarden_tb,100,jaelondarden_tb,wr,illness,out
11,darrellhenderson_lar,darrellhenderson_lar,100,darrellhenderson_lar,rb,knee mcl,ir
13,ryangriffin_nyj,ryangriffin_nyj,100,ryangriffin_nyj,te,knee,out


In [30]:
#injured_players_list.to_csv(cur_dir+'/other_data/injured_player_pool_week10.csv', index=False)

### Use the injury players data to subset our Player Pool

In [31]:
#inj = pd.read_csv(cur_dir+'/other_data/injured_player_pool_week10.csv')

rush_std_new = rush_std_new[~rush_std_new['player_team'].isin(list(set(injured_players_list['match_name'])))]
qb_standard_new = qb_standard_new[~qb_standard_new['player_team'].isin(list(set(injured_players_list['match_name'])))]
oline_standard_new = oline_standard_new[~oline_standard_new['player_team'].isin(list(set(injured_players_list['match_name'])))]
rec_std_new = rec_std_new[~rec_std_new['player_team'].isin(list(set(injured_players_list['match_name'])))]
def_standard_new = def_standard_new[~def_standard_new['player_team'].isin(list(set(injured_players_list['match_name'])))]

In [32]:
rush_std_new=rush_std_new.apply(pd.to_numeric, errors='ignore')
rush_std_new = rush_std_new[rush_std_new['position'].isin(['hb','fb','qb'])]

qb_standard_new=qb_standard_new.apply(pd.to_numeric, errors='ignore')
qb_standard_new = qb_standard_new[qb_standard_new['position'].isin(['qb'])]

rec_std_new=rec_std_new.apply(pd.to_numeric, errors='ignore')
rec_std_new = rec_std_new[rec_std_new['position'].isin(['wr','te','hb'])]

oline_standard_new=oline_standard_new.apply(pd.to_numeric, errors='ignore')
oline_standard_new = oline_standard_new[oline_standard_new['position'].isin(['t','g','ol'])]

In [33]:
oline_standard_new.head().T

,0,1,2,3,4
unique_id,taylorlewan_ten_2021_17,treypipkins_lac_2021_17,coreylinsley_lac_2021_17,camerving_car_2021_17,patelflein_car_2021_17
player_id,taylorlewan_ten_2021,treypipkins_lac_2021,coreylinsley_lac_2021,camerving_car_2021,patelflein_car_2021
unique_team_id,ten_2021_17,lac_2021_17,lac_2021_17,car_2021_17,car_2021_17
team_id_impute,ten_2021,lac_2021,lac_2021,car_2021,car_2021
player_name,taylorlewan,treypipkins,coreylinsley,camerving,patelflein
player_team,taylorlewan_ten,treypipkins_lac,coreylinsley_lac,camerving_car,patelflein_car
player,taylorlewan,treypipkins,coreylinsley,camerving,patelflein
position,t,t,ol,t,ol
team_name,ten,lac,lac,car,car
player_game_count,1,1,1,1,1


In [34]:
#This cell is foor exporting the player pool to for spot testing purposes#
rush_std_new = rush_std_new.sort_values(by=["team_name","week"], ascending=[True, False])
qb_standard_new = qb_standard_new.sort_values(by=["week"], ascending=[False])
rec_std_new = rec_std_new.sort_values(by=["team_name","week"], ascending=[True, False])
oline_standard_new = oline_standard_new.sort_values(by=["team_name","week"], ascending=[True, False])
def_standard_new = def_standard_new.sort_values(by=["team_name","week"], ascending=[True, False])

oline_standard_new['unique_id']=oline_standard_new['unique_id'].str.replace('2020.0', '2020', regex=True)
oline_standard_new['player_id']=oline_standard_new['player_id'].str.replace('2020.0', '2020', regex=True)
oline_standard_new['unique_team_id']=oline_standard_new['unique_team_id'].str.replace('2020.0', '2020', regex=True)
oline_standard_new['team_id_impute']=oline_standard_new['team_id_impute'].str.replace('2020.0', '2020', regex=True)


rush_std_new.to_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/rush_std_new_pp_w'+cur_week+'.csv', index=False)
qb_standard_new.to_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/qb_standard_new_pp_w'+cur_week+'.csv', index=False)
oline_standard_new.to_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/oline_standard_new_pp_w'+cur_week+'.csv', index=False)
rec_std_new.to_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/rec_std_new_pp_w'+cur_week+'.csv', index=False)
def_standard_new.to_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/def_standard_new_pp_w'+cur_week+'.csv', index=False)

In [35]:
rush_std_new_prev = rush_std_new
qb_standard_new_prev = qb_standard_new
oline_standard_new_prev = oline_standard_new
rec_std_new_prev = rec_std_new
def_standard_new_prev = def_standard_new

In [36]:
tgs_2019 = pd.read_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/team_game_summaries_2021_w'+cur_week+'.csv')

In [37]:
tgs_2019 = tgs_2019[~tgs_2019['week'].isnull()]
#tgs_2019 = tgs_2019[tgs_2019['week'] == cur_week]
tgs_2019=tgs_2019.drop_duplicates(subset=['team','year'], keep='last').assign(week=cur_week)


##  functions to clean and scrub data by source  ##
def clean_pff_team_summ(df):
##  basic scrubbing to clean data ##

    ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team'].map(cleaning_dicts.clean_team_pff_full).fillna(df['team'])
    df['opponent_name'] = df['opponent'].map(cleaning_dicts.clean_team_pff_opp).fillna(df['opponent'])
    
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)


    ##  create our unique ids  ##
    df.insert(0, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(2, "opponent_id", (df['opponent_name']+'_'+df['year']+'_'+df['week']))


    return df
    
tgs_clean = clean_pff_team_summ(tgs_2019)


tgs_clean['wl_int'] = ''
tgs_clean['fav_homeoraway'] = ''
tgs_clean['team_favored_or_not'] = ''
tgs_clean['ats_w'] = ''
tgs_clean['ats_l'] = ''


tgs_clean = tgs_clean.sort_values(by=["team_name","week"], ascending=[True, False])
tgs_clean.to_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/tgs_clean_pp_w'+cur_week+'.csv', index=False)

In [38]:
tgs_2019 = pd.read_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/team_game_summaries_2021_w'+cur_week+'.csv')

tgs_2019 = tgs_2019[~tgs_2019['week'].isnull()]
#tgs_2019 = tgs_2019[tgs_2019['week'] == cur_week]
tgs_2019=tgs_2019.drop_duplicates(subset=['team','year'], keep='last').assign(week=cur_week)

In [39]:
tgs_2019.head().T

,14,29,44,59,74
team,arizona-cardinals,atlanta-falcons,baltimore-ravens,buffalo-bills,carolina-panthers
year,2021,2021,2021,2021,2021
week,17,17,17,17,17
home_or_away,NaN,NaN,@,@,NaN
team_num,1,2,3,4,5
game_id,19833,19835,19841,19838,19836
opponent,Indianapolis Colts,Detroit Lions,Cincinnati Bengals,New England Patriots,Tampa Bay Buccaneers
date,12/25,12/26,12/26,12/26,12/26
time,8:15pm,1:00pm,1:00pm,1:00pm,1:00pm
wl,L,W,L,W,L


In [40]:
fo_2021 = pd.read_csv(cur_dir+'/football_outsiders/fo_final.csv')

prev = str(cur_week_int-1)

fo_2021 = fo_2021[~fo_2021['week'].isnull()]
fo_2021.tail()

,team_id,year,week,team,w-l,off_dvoa_rank,off_dvoa,off_weight_dvoa_rank,off_weight_dvoa,pass_off_rank,...,rush_off_rank,rush_off,def_dvoa_rank,def_dvoa,def_weight_dvoa_rank,def_weight_dvoa,pass_def_rank,pass_def,rush_def_rank,rush_def
507,pit_2021_16,2021,16,pit,7-7-1,23,-8.2,24,-11.8,21,...,28,-17.9,22,3.4,25,5.7,18,5.7,30,0.7
508,mia_2021_16,2021,16,mia,8-7,25,-11.9,23,-11.4,23,...,30,-26.0,8,-6.0,7,-10.5,6,-2.9,13,-10.8
509,jax_2021_16,2021,16,jax,2-13,27,-14.1,26,-15.8,28,...,18,-8.3,30,10.9,26,7.0,31,29.2,16,-10.0
510,det_2021_16,2021,16,det,2-12-1,29,-19.4,29,-22.2,29,...,26,-16.0,29,10.8,29,9.0,26,20.5,28,-1.0
511,car_2021_16,2021,16,car,5-10,31,-22.5,31,-27.3,32,...,19,-10.6,12,-1.7,21,1.4,9,2.0,20,-5.7


In [41]:
fo_2021 = pd.read_csv(cur_dir+'/football_outsiders/fo_final.csv')

prev = str(cur_week_int-1)

fo_2021 = fo_2021[~fo_2021['week'].isnull()]
#tgs_2019 = tgs_2019[tgs_2019['week'] == cur_week]
fo_2021_new=fo_2021.drop_duplicates(subset=['team','year'], keep='last').assign(week=cur_week)
fo_2021_new['team_id']=fo_2021_new['team_id'].str.replace("2021_"+prev, str("2021_"+cur_week))

fo_2021_new = fo_2021_new.sort_values(by=["team","week"], ascending=[True, False])
fo_2021_new.to_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/fo_clean_pp_w'+cur_week+'.csv', index=False)

In [42]:
fo_2021_new

,team_id,year,week,team,w-l,off_dvoa_rank,off_dvoa,off_weight_dvoa_rank,off_weight_dvoa,pass_off_rank,...,rush_off_rank,rush_off,def_dvoa_rank,def_dvoa,def_weight_dvoa_rank,def_weight_dvoa,pass_def_rank,pass_def,rush_def_rank,rush_def
487,ari_2021_17,2021,17,ari,10-5,16,3.4,19,-2.9,8,...,22,-12.9,5,-9.7,11,-7.9,5,-3.7,5,-18.1
493,atl_2021_17,2021,17,atl,7-8,28,-18.3,28,-17.8,24,...,31,-31.6,31,12.9,30,11.5,29,25.7,25,-3.4
503,bal_2021_17,2021,17,bal,8-7,15,4.3,17,-0.3,15,...,8,-1.7,28,10.7,31,12.9,30,28.3,6,-17.4
482,buf_2021_17,2021,17,buf,9-6,6,11.7,4,15.5,6,...,14,-3.6,2,-15.9,4,-12.7,2,-20.0,14,-10.8
511,car_2021_17,2021,17,car,5-10,31,-22.5,31,-27.3,32,...,19,-10.6,12,-1.7,21,1.4,9,2.0,20,-5.7
492,chi_2021_17,2021,17,chi,5-10,26,-11.9,25,-12.0,26,...,21,-12.6,23,3.4,23,4.0,24,9.7,24,-3.8
505,cin_2021_17,2021,17,cin,9-6,19,-1.1,15,1.3,18,...,20,-11.5,14,-0.4,15,-1.7,22,9.3,7,-15.9
486,cle_2021_17,2021,17,cle,7-8,14,5.0,18,-1.6,19,...,2,10.4,17,0.7,18,-0.9,19,8.0,18,-9.0
483,dal_2021_17,2021,17,dal,11-4,8,10.6,10,6.0,7,...,16,-4.2,1,-16.6,1,-21.1,1,-22.6,19,-7.1
485,den_2021_17,2021,17,den,7-8,12,5.9,11,5.3,13,...,11,-2.5,19,1.3,13,-2.2,16,5.0,23,-4.1


In [43]:
import pandas as pd
import numpy as np
import re
from string import ascii_letters, digits
import cleaning_dicts
import os

#print(cleaning_dicts.clean_team_pff)
print(cleaning_dicts.clean_team_pff_full)

#cur_dir = "C:/Users/booth/Documents/spreads/spread_2020"
cur_dir = '/media/tom/Windows/Users/booth/Documents/spreads/spreads_2021/'



qb_standard_2019 = pd.read_csv(cur_dir+'historic_data'+'/qb_standard_hist_w'+cur_week+'.csv')
oline_2019 = pd.read_csv(cur_dir+'historic_data'+'/ol_standard_hist_w'+cur_week+'.csv')
#qb_adj_2019 = pd.read_csv(cur_dir+'historic_data'+'/qb_adjusted_hist_16.csv')
rush_std_2019 = pd.read_csv(cur_dir+'historic_data'+'/rush_standard_hist_w'+cur_week+'.csv')
rec_std_2019 = pd.read_csv(cur_dir+'historic_data'+'/rec_standard_hist_w'+cur_week+'.csv')
def_standard_2019 = pd.read_csv(cur_dir+'historic_data'+'/def_standard_hist_w'+cur_week+'.csv')

qb_standard_2019 = qb_standard_2019[~qb_standard_2019['player'].isnull()]
oline_2019 = oline_2019[~oline_2019['player'].isnull()]
#qb_adj_2019 = qb_adj_2019[~qb_adj_2019['player'].isnull()]
rush_std_2019 = rush_std_2019[~rush_std_2019['player'].isnull()]
rec_std_2019 = rec_std_2019[~rec_std_2019['player'].isnull()]
def_standard_2019 = def_standard_2019[~def_standard_2019['player'].isnull()]



qb_standard_2019 = qb_standard_2019[qb_standard_2019['year'] == 2021]
oline_2019 = oline_2019[oline_2019['year'] == 2021]
#qb_adj_2019 = qb_adj_2019[~qb_adj_2019['player'].isnull()]
rush_std_2019 = rush_std_2019[rush_std_2019['year'] == 2021]
rec_std_2019 = rec_std_2019[rec_std_2019['year'] == 2021]
def_standard_2019 = def_standard_2019[def_standard_2019['year'] == 2021]

qb_standard_2019

qb_standard_new = qb_standard_2019[qb_standard_2019['week'] >= cur_week_int-2]
oline_new = oline_2019[oline_2019['week'] >=  cur_week_int-2]
#qb_adj_new = qb_adj_2019[qb_adj_2019['week'] >=  cur_week_int-2]
rush_std_new = rush_std_2019[rush_std_2019['week'] >=  cur_week_int-2]
rec_std_new = rec_std_2019[rec_std_2019['week'] >=  cur_week_int-2]
def_standard_new = def_standard_2019[def_standard_2019['week'] >=  cur_week_int-2]

qb_standard_nw=qb_standard_new.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
oline_standard_nw=oline_new.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
#qb_adj_nw=qb_adj_new.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
rush_std_nw=rush_std_new.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
rec_std_nw=rec_std_new.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
def_standard_nw=def_standard_new.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)

qb_standard_nw=qb_standard_2019.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
oline_standard_nw=oline_2019.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
#qb_adj_nw=qb_adj_2019.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
rush_std_nw=rush_std_2019.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
rec_std_nw=rec_std_2019.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)
def_standard_nw=def_standard_2019.drop_duplicates(subset=['player','year'], keep='last').assign(plyr_number=cur_week)

qb_standard_nw.head(n=5)

qb_standard_new

def df_nans(nData):
    a = np.full([nData.shape[0], nData.shape[1]], 0)
    np_df = pd.DataFrame(a, columns=nData.columns)
    ids = nData.iloc[:,:8].reset_index(drop=True)
    other_cols = np_df.iloc[:,8:]
    df = pd.concat([ids, other_cols], axis=1)
    return df

qb_standard_new = df_nans(qb_standard_nw)
oline_standard_new = df_nans(oline_standard_nw)
rush_std_new = df_nans(rush_std_nw)
rec_std_new = df_nans(rec_std_nw)
def_standard_new = df_nans(def_standard_nw)
#qb_adj_new = df_nans(qb_adj_nw)


def clean_str_nan(nData):
    nData.replace(["NaN", 'NaT'], np.nan, inplace = True)
    nData=nData[pd.notnull(nData['player'])]
    #nData=nData.dropna(thresh=10)
    nData['year'] = cur_year_int
    nData['week'] = cur_week_int
    nData['plyr_number'] = cur_week_int
    nData['year'] = nData['year'].astype(int)
    nData['week'] = nData['week'].astype(int)
    return nData

rush_std_new = clean_str_nan(rush_std_new)
qb_standard_new =clean_str_nan(qb_standard_new)
oline_standard_new=clean_str_nan(oline_standard_new)
#qb_adj_new=clean_str_nan(qb_adj_new)
rec_std_new=clean_str_nan(rec_std_new)
def_standard_new=clean_str_nan(def_standard_new)


rush_std_new.drop('player_id', axis=1, inplace=True)
qb_standard_new.drop('player_id', axis=1, inplace=True)
oline_standard_new.drop('player_id', axis=1, inplace=True)
rec_std_new.drop('player_id', axis=1, inplace=True)
def_standard_new.drop('player_id', axis=1, inplace=True)

rush_std_new.head()

def clean_name(df):
    result = ''.join(c for c in df['player'] if c.isalpha())
    return result.lower()



#C:\Users\booth\Downloads\nfl_spreads-master\nfl_spreads-master\other_data
nfl_injuries = pd.read_csv(cur_dir+'nfl_injuries/nfl_injuries_w'+cur_week+'.csv')
nfl_injuries.columns = ['Player','Tm','Pos','Type','Class']
nfl_injuries.columns = [x.lower() for x in nfl_injuries.columns]
nfl_injuries = nfl_injuries[nfl_injuries['tm'] != 'FA']
nfl_injuries.drop_duplicates(subset=['player','tm','pos'], keep='first', inplace=True)
nfl_injuries['player'] = nfl_injuries.apply(lambda df: clean_name(df), axis=1)

def clean_non_player_dfs(df):
##  basic scrubbing to clean data ##
    df=df.apply(lambda x: x.astype(str).str.lower())
    df=df.replace('-','', regex=True)
    df['player']=df['player'].replace(' ','', regex=True)
    df['tm']=df['tm'].replace(' ','', regex=True)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['tm'].map(cleaning_dicts.clean_team_pfr).fillna(df['tm'])
    df['pos'] = df['pos'].map(cleaning_dicts.pos_dict).fillna(df['pos'])

    df.insert(0, "player_team", (df['player']+'_'+df['team_name']))
    return df

nfl_injuries = clean_non_player_dfs(nfl_injuries)

nfl_injuries['class']=nfl_injuries['class'].str.replace('pupr','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('nfir','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('nfia','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('inactive','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reservecovid19','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reservesus','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reservednr','out')
nfl_injuries['class']=nfl_injuries['class'].str.replace('reserveret','out')

nfl_injuries.head(n=50)



rush_std_new.head()

qb_standard_new['player'] = qb_standard_new.apply(lambda df: clean_name(df), axis=1)
#qb_adj_new['player'] = qb_adj_new.apply(lambda df: clean_name(df), axis=1)
oline_standard_new['player'] = oline_standard_new.apply(lambda df: clean_name(df), axis=1)
rush_std_new['player'] = rush_std_new.apply(lambda df: clean_name(df), axis=1)
rec_std_new['player'] = rec_std_new.apply(lambda df: clean_name(df), axis=1)
def_standard_new['player'] = def_standard_new.apply(lambda df: clean_name(df), axis=1)

oline_standard_new




def clean_new_week(df):
##  basic scrubbing to clean data ##
    df['year'] = df['year'].apply(str)
    df=df.apply(lambda x: x.astype(str).str.lower())
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)

    ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(cleaning_dicts.pos_dict).fillna(df['position'])

    ##  create our unique ids  ##
    df.insert(0, "unique_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['plyr_number']))
    df.insert(1, "player_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(2, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['plyr_number']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(4, "player_name", (df['player']))
    df.insert(5, "player_team", (df['player']+'_'+df['team_name']))
    return df

rush_std_new = clean_new_week(rush_std_new)
qb_standard_new =clean_new_week(qb_standard_new)
oline_standard_new=clean_new_week(oline_standard_new)
rec_std_new=clean_new_week(rec_std_new)
def_standard_new=clean_new_week(def_standard_new)



rush_std_new.head().T

## Add this for dummy adj comp percentage columns
#qb_standard_new['adj_perc'] = 0
#qb_standard_new['qb_adjform'] = 0



from fuzzywuzzy import fuzz

def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = 0
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

def fuzzy_inj_match(df):
    # List for dicts for easy dataframe creation
    dict_list = []
    # iterating over our players without salaries found above
    for name in df.player_team:
        # Use our method to find best match, we can set a threshold here
        match = match_name(name, nfl_injuries.player_team, 85)

        # New dict for storing data
        dict_ = {}
        dict_.update({"player_name" : name})
        dict_.update({"match_name" : match[0]})
        dict_.update({"score" : match[1]})
        dict_list.append(dict_)
    return pd.DataFrame(dict_list)
        #return pd.merge(df, merge_table, left_on='unique_player_id', right_on='player_name', how='left')

# Display results
rush_std_new_match = fuzzy_inj_match(rush_std_new)
qb_standard_new_match = fuzzy_inj_match(qb_standard_new)
oline_standard_new_match = fuzzy_inj_match(oline_standard_new)
rec_std_new_match = fuzzy_inj_match(rec_std_new)
def_standard_new_match = fuzzy_inj_match(def_standard_new)

rush_std_new_match = rush_std_new_match[rush_std_new_match['score'] != 0]
qb_standard_new_match = qb_standard_new_match[qb_standard_new_match['score'] != 0]
oline_standard_new_match = oline_standard_new_match[oline_standard_new_match['score'] != 0]
rec_std_new_match = rec_std_new_match[rec_std_new_match['score'] != 0]
def_standard_new_match = def_standard_new_match[def_standard_new_match['score'] != 0]

def_standard_new_match.head(n=20)

nfl_injuries = nfl_injuries[['player_team','pos','type','class']]
rush_std_new_match = pd.merge(rush_std_new_match, nfl_injuries, left_on='player_name', right_on='player_team', how='left')
qb_standard_new_match = pd.merge(qb_standard_new_match, nfl_injuries, left_on='player_name', right_on='player_team', how='left')
oline_standard_new_match = pd.merge(oline_standard_new_match, nfl_injuries, left_on='player_name', right_on='player_team', how='left')
rec_std_new_match = pd.merge(rec_std_new_match, nfl_injuries, left_on='player_name', right_on='player_team', how='left')
def_standard_new_match = pd.merge(def_standard_new_match, nfl_injuries, left_on='player_name', right_on='player_team', how='left')


rush_std_new_match.head()

rush_std_new_match = rush_std_new_match.loc[(rush_std_new_match['class'] == 'ir') | (rush_std_new_match['class'] == 'out') | (rush_std_new_match['class'] == 'bench') | (rush_std_new_match['class'] == 'doubtful') | (rush_std_new_match['class'] == 'cut')]
qb_standard_new_match = qb_standard_new_match.loc[(qb_standard_new_match['class'] == 'ir') | (qb_standard_new_match['class'] == 'out') | (qb_standard_new_match['class'] == 'bench') | (qb_standard_new_match['class'] == 'doubtful') | (qb_standard_new_match['class'] == 'cut')]
oline_standard_new_match = oline_standard_new_match.loc[(oline_standard_new_match['class'] == 'ir') | (oline_standard_new_match['class'] == 'out') | (oline_standard_new_match['class'] == 'bench') | (oline_standard_new_match['class'] == 'doubtful') | (oline_standard_new_match['class'] == 'cut')]
rec_std_new_match = rec_std_new_match.loc[(rec_std_new_match['class'] == 'ir') | (rec_std_new_match['class'] == 'out') | (rec_std_new_match['class'] == 'bench') | (rec_std_new_match['class'] == 'doubtful') | (rec_std_new_match['class'] == 'cut')]
def_standard_new_match = def_standard_new_match.loc[(def_standard_new_match['class'] == 'ir') | (def_standard_new_match['class'] == 'out') | (def_standard_new_match['class'] == 'bench') | (def_standard_new_match['class'] == 'doubtful') | (def_standard_new_match['class'] == 'cut')]

rec_std_new_match.head(n=5)

injured_players_list= pd.concat([rush_std_new_match,qb_standard_new_match,oline_standard_new_match,rec_std_new_match,def_standard_new_match], axis=0)

injured_players_list.head(n=100)

#injured_players_list.to_csv(cur_dir+'/other_data/injured_player_pool_week10.csv', index=False)

### Use the injury players data to subset our Player Pool

#inj = pd.read_csv(cur_dir+'/other_data/injured_player_pool_week10.csv')

rush_std_new = rush_std_new[~rush_std_new['player_team'].isin(list(set(injured_players_list['match_name'])))]
qb_standard_new = qb_standard_new[~qb_standard_new['player_team'].isin(list(set(injured_players_list['match_name'])))]
oline_standard_new = oline_standard_new[~oline_standard_new['player_team'].isin(list(set(injured_players_list['match_name'])))]
rec_std_new = rec_std_new[~rec_std_new['player_team'].isin(list(set(injured_players_list['match_name'])))]
def_standard_new = def_standard_new[~def_standard_new['player_team'].isin(list(set(injured_players_list['match_name'])))]

rush_std_new=rush_std_new.apply(pd.to_numeric, errors='ignore')
rush_std_new = rush_std_new[rush_std_new['position'].isin(['hb','fb','qb'])]

qb_standard_new=qb_standard_new.apply(pd.to_numeric, errors='ignore')
qb_standard_new = qb_standard_new[qb_standard_new['position'].isin(['qb'])]

rec_std_new=rec_std_new.apply(pd.to_numeric, errors='ignore')
rec_std_new = rec_std_new[rec_std_new['position'].isin(['wr','te','hb'])]

oline_standard_new=oline_standard_new.apply(pd.to_numeric, errors='ignore')
oline_standard_new = oline_standard_new[oline_standard_new['position'].isin(['t','g','ol'])]

oline_standard_new.head().T

#This cell is foor exporting the player pool to for spot testing purposes#
rush_std_new = rush_std_new.sort_values(by=["team_name","week"], ascending=[True, False])
qb_standard_new = qb_standard_new.sort_values(by=["week"], ascending=[False])
rec_std_new = rec_std_new.sort_values(by=["team_name","week"], ascending=[True, False])
oline_standard_new = oline_standard_new.sort_values(by=["team_name","week"], ascending=[True, False])
def_standard_new = def_standard_new.sort_values(by=["team_name","week"], ascending=[True, False])

oline_standard_new['unique_id']=oline_standard_new['unique_id'].str.replace('2020.0', '2020', regex=True)
oline_standard_new['player_id']=oline_standard_new['player_id'].str.replace('2020.0', '2020', regex=True)
oline_standard_new['unique_team_id']=oline_standard_new['unique_team_id'].str.replace('2020.0', '2020', regex=True)
oline_standard_new['team_id_impute']=oline_standard_new['team_id_impute'].str.replace('2020.0', '2020', regex=True)

p1 = list(set(rush_std_new['player_id']))
p2 = list(set(qb_standard_new['player_id']))
p3 = list(set(oline_standard_new['player_id']))
p4 = list(set(rec_std_new['player_id']))
p5 = list(set(def_standard_new['player_id']))

p1_list = list(set(rush_std_new['player_id']) - set(rush_std_new_prev['player_id']))
p2_list = list(set(qb_standard_new['player_id']) - set(qb_standard_new_prev['player_id']))
p3_list = list(set(oline_standard_new['player_id']) - set(oline_standard_new_prev['player_id']))
p4_list = list(set(rec_std_new['player_id']) - set(rec_std_new_prev['player_id']))
p5_list = list(set(def_standard_new['player_id']) - set(def_standard_new_prev['player_id']))

rush_std_new = rush_std_new[rush_std_new['player_id'].isin(p1_list)]
qb_standard_new = qb_standard_new[qb_standard_new['player_id'].isin(p2_list)]
oline_standard_new = oline_standard_new[oline_standard_new['player_id'].isin(p3_list)]
rec_std_new = rec_std_new[rec_std_new['player_id'].isin(p4_list)]
def_standard_new = def_standard_new[def_standard_new['player_id'].isin(p5_list)]

rush_std_new.to_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/rush_std_new_pp_return_w'+cur_week+'.csv', index=False)
qb_standard_new.to_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/qb_standard_new_pp_return_w'+cur_week+'.csv', index=False)
oline_standard_new.to_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/oline_standard_new_pp_return_w'+cur_week+'.csv', index=False)
rec_std_new.to_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/rec_std_new_pp_return_w'+cur_week+'.csv', index=False)
def_standard_new.to_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/def_standard_new_pp_return_w'+cur_week+'.csv', index=False)




{'arizona-cardinals': 'ari', 'atlanta-falcons': 'atl', 'baltimore-ravens': 'bal', 'buffalo-bills': 'buf', 'carolina-panthers': 'car', 'chicago-bears': 'chi', 'cincinnati-bengals': 'cin', 'cleveland-browns': 'cle', 'dallas-cowboys': 'dal', 'denver-broncos': 'den', 'detroit-lions': 'det', 'green-bay-packers': 'gb', 'houston-texans': 'hou', 'indianapolis-colts': 'ind', 'jacksonville-jaguars': 'jax', 'kansas-city-chiefs': 'kc', 'los-angeles-chargers': 'lac', 'los-angeles-rams': 'lar', 'miami-dolphins': 'mia', 'minnesota-vikings': 'min', 'new-england-patriots': 'ne', 'new-orleans-saints': 'no', 'new-york-giants': 'nyg', 'new-york-jets': 'nyj', 'oakland-raiders': 'oak', 'philadelphia-eagles': 'phi', 'pittsburgh-steelers': 'pit', 'san-francisco-49ers': 'sf', 'seattle-seahawks': 'sea', 'tampa-bay-buccaneers': 'tb', 'tennessee-titans': 'ten', 'washington-redskins': 'was', 'washington-football-team': 'was', 'las-vegas-raiders': 'lv'}


In [44]:
rush_std_new

,unique_id,player_id,unique_team_id,team_id_impute,player_name,player_team,player,position,team_name,player_game_count,...,touchdowns,yards,yards_after_contact,yco_attempt,ypa,yprr,zone_attempts,week,year,plyr_number
74,chrisstreveler_ari_2021_17,chrisstreveler_ari_2021,ari_2021_17,ari_2021,chrisstreveler,chrisstreveler_ari,chrisstreveler,qb,ari,1,...,0,0,0,0,0,0,0,17,2021,17
136,enobenjamin_ari_2021_17,enobenjamin_ari_2021,ari_2021_17,ari_2021,enobenjamin,enobenjamin_ari,enobenjamin,hb,ari,1,...,0,0,0,0,0,0,0,17,2021,17
66,feleipefranks_atl_2021_17,feleipefranks_atl_2021,atl_2021_17,atl_2021,feleipefranks,feleipefranks_atl,feleipefranks,qb,atl,1,...,0,0,0,0,0,0,0,17,2021,17
81,keithsmith_atl_2021_17,keithsmith_atl_2021,atl_2021_17,atl_2021,keithsmith,keithsmith_atl,keithsmith,fb,atl,1,...,0,0,0,0,0,0,0,17,2021,17
109,waynegallman_atl_2021_17,waynegallman_atl_2021,atl_2021_17,atl_2021,waynegallman,waynegallman_atl,waynegallman,hb,atl,1,...,0,0,0,0,0,0,0,17,2021,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,blainegabbert_tb_2021_17,blainegabbert_tb_2021,tb_2021_17,tb_2021,blainegabbert,blainegabbert_tb,blainegabbert,qb,tb,1,...,0,0,0,0,0,0,0,17,2021,17
110,khariblasingame_ten_2021_17,khariblasingame_ten_2021,ten_2021_17,ten_2021,khariblasingame,khariblasingame_ten,khariblasingame,fb,ten,1,...,0,0,0,0,0,0,0,17,2021,17
159,loganwoodside_ten_2021_17,loganwoodside_ten_2021,ten_2021_17,ten_2021,loganwoodside,loganwoodside_ten,loganwoodside,qb,ten,1,...,0,0,0,0,0,0,0,17,2021,17
145,wendellsmallwood_was_2021_17,wendellsmallwood_was_2021,was_2021_17,was_2021,wendellsmallwood,wendellsmallwood_was,wendellsmallwood,hb,was,1,...,0,0,0,0,0,0,0,17,2021,17
